# NOT WORKING YET

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  

%matplotlib inline

from pathlib import Path
import sys
sys.path.append("../../")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tqdm.auto import tqdm

import captchanet

data_dir = Path('/home/hadim/.data/Neural_Network/captchanet')

dataset_dir = data_dir / 'dataset_v6'
train_data_dir = dataset_dir / 'training'
val_data_dir = dataset_dir / 'validation'

model_dir = data_dir / 'model'
model_name = 'v6'
model_path = model_dir / model_name

tokenizer_path = model_path / "tokenizer.json"

# Get tokenizer
with open(tokenizer_path) as f:
  #tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(f.read())
  from keras_preprocessing import text
  tokenizer = text.tokenizer_from_json(f.read())
  
model = tf.keras.models.load_model(str(model_path / 'model'))

In [2]:
model_image_size = (224, 224)
n  = 2
fnames = [str(p) for p in val_data_dir.glob("*.tfrecord")]
dataset = tf.data.TFRecordDataset(fnames)
dataset = dataset.map(map_func=captchanet.decode_data(tokenizer, image_size=model_image_size, max_len_word=10, input_as_dict=True))
dataset = dataset.shuffle(1024)
dataset = dataset.batch(n)
data = [d for d in dataset.take(1)][0]

images = data['image']
labels = data['label']
labels = model(images)

W0713 19:23:19.385547 139967693227840 deprecation.py:323] From /home/hadim/local/conda/envs/captchanet/lib/python3.7/site-packages/tensorflow/python/ops/image_ops_impl.py:1511: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [13]:
# Define a model that do image preprocessing before inference.
class CaptchaSolverInferenceModel(tf.keras.Model):
  
  def __init__(self, captchanet_model, **kwargs):
    super().__init__(**kwargs)
    self.captchanet_model = captchanet_model

  @tf.function(input_signature=[tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8)])
  def solve(self, images):
    images = tf.cast(images, tf.float32)
    
    # Normalize images
    #images = tf.image.per_image_standardization(images)
    #stddev = tf.math.reduce_std(images, axis=0)
    #mean = tf.math.reduce_mean(images, axis=0)
    #images = (images - mean) / stddev
    
    images = tf.image.resize(images, (224, 224))
    
    labels = model(images)
    #labels = tf.argmax(labels, axis=2)
    # TODO: find a way to integrate the tokenizer.
    
    # Freeze the tokenizer
    #n_letter = max(tokenizer.index_word.keys()) + 1
    #index = [''] * n_letter
    #for k, v in tokenizer.index_word.items():
    #  index[k] = v
    #index = tf.constant(index, dtype=tf.string)
        
    # Decode labels
    #decoder = lambda x: tf.gather_nd(index, tf.expand_dims(x, axis=-1))
    #words = tf.map_fn(decoder, labels, dtype=tf.string)
    #joiner = lambda x: tf.strings.reduce_join(x, axis=0)
    #words = tf.map_fn(joiner, words, dtype=tf.string)

    return images

model = tf.saved_model.load(str(model_path / 'model'))
inference_model = CaptchaSolverInferenceModel(model)
inference_model_path = model_path.parent / 'test_model_tfjs'
tf.saved_model.save(inference_model, str(inference_model_path))

W0713 19:29:39.462048 139967693227840 saved_model.py:758] Skipping full serialization of Keras model <__main__.CaptchaSolverInferenceModel object at 0x7f4b37e6ccf8>, because its inputs are not defined.


In [14]:
import tensorflowjs as tfjs
tfjs.converters.convert_tf_saved_model(str(inference_model_path), '/home/hadim/testmodel')

ValueError: Input 0 of node Func/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/keras_layer/StatefulPartitionedCall/StatefulPartitionedCall_1/input/_2007 was passed float from Func/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/keras_layer/StatefulPartitionedCall/input/_1139:0 incompatible with expected resource.

In [12]:
model(images)

<tf.Tensor: id=199896, shape=(2, 10, 60), dtype=float32, numpy=
array([[[2.9817571e-07, 3.5087726e-04, 3.0216272e-05, ...,
         3.7906871e-06, 6.3952053e-04, 6.7858683e-04],
        [7.6777829e-09, 5.2266045e-05, 6.4965086e-08, ...,
         3.6354169e-05, 3.2530249e-07, 6.5024155e-06],
        [1.5669572e-06, 6.3543380e-03, 8.5173924e-05, ...,
         4.7941823e-05, 1.5778298e-03, 3.6667331e-04],
        ...,
        [9.9802303e-01, 4.3144079e-05, 2.9469247e-05, ...,
         2.3697708e-05, 6.6060158e-05, 6.6571702e-06],
        [9.9970835e-01, 1.2790371e-05, 1.8501147e-06, ...,
         3.0394544e-06, 8.0580976e-06, 7.0386136e-06],
        [9.9997234e-01, 3.8388180e-07, 4.0560349e-07, ...,
         2.2472686e-07, 3.0780257e-06, 1.2501309e-06]],

       [[5.1425661e-05, 1.7609533e-02, 3.8806025e-03, ...,
         1.3713936e-03, 9.5771207e-04, 1.6140189e-03],
        [1.9176459e-06, 5.2574975e-05, 1.7011825e-04, ...,
         2.0871112e-04, 1.6116243e-04, 3.8373042e-03],
        [